In [2]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

%matplotlib inline

In [3]:
# Here you run your code to explore the data
conn = sqlite3.connect('data\movies.db')
cur = conn.cursor()

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [5]:
# Here you run your code to clean the data

First joining `tn_movie_budgets` with `imdb_movie_basics`

In [6]:
cur.execute("""SELECT b.movie, b.production_budget, b.domestic_gross, b.worldwide_gross, m.genres 
               FROM tn_movie_budgets b
               JOIN imdb_title_basics m
               ON b.movie = m.primary_title
               GROUP BY movie
               ORDER BY worldwide_gross DESC;""")
gross_df = pd.DataFrame(cur.fetchall())
gross_df.columns = [x[0] for x in cur.description]
gross_df

,movie,production_budget,domestic_gross,worldwide_gross,genres
0,Duma,"$12,000,000","$870,067","$994,790","Biography,Crime,Documentary"
1,Insidious,"$1,500,000","$54,009,150","$99,870,886","Horror,Mystery,Thriller"
2,Hellboy,"$60,000,000","$59,623,958","$99,823,958","Action,Adventure,Fantasy"
3,Alpha,"$51,000,000","$35,851,379","$99,624,873","Adventure,Drama,Family"
4,Hitman,"$24,000,000","$39,687,694","$99,135,571",Action
...,...,...,...,...,...
2307,A Fine Step,"$1,000,000",$0,$0,Drama
2308,8 Days,"$2,500,000",$0,$0,"Drama,Thriller"
2309,40 Weeks,"$900,000",$0,$0,Documentary
2310,1982,"$1,000,000",$0,$0,Documentary


when splitting genres, be careful with economic data because I don't want to add up the duplicate values for each movie and then say "this was the total budget for that movie", etc

Creating IMDB ratings data frame

In [24]:
cur.execute("""SELECT m.primary_title, r.averagerating, r.numvotes, m.genres
               FROM imdb_title_basics m
               JOIN imdb_title_ratings r
               USING(tconst);""")
ratings_df = pd.DataFrame(cur.fetchall())
ratings_df.columns = [x[0] for x in cur.description]
ratings_df

,primary_title,averagerating,numvotes,genres
0,Sunghursh,7.0,77,"Action,Crime,Drama"
1,One Day Before the Rainy Season,7.2,43,"Biography,Drama"
2,The Other Side of the Wind,6.9,4517,Drama
3,Sabse Bada Sukh,6.1,13,"Comedy,Drama"
4,The Wandering Soap Opera,6.5,119,"Comedy,Drama,Fantasy"
...,...,...,...,...
73851,Diabolik sono io,6.2,6,Documentary
73852,Sokagin Çocuklari,8.7,136,"Drama,Family"
73853,Albatross,8.5,8,Documentary
73854,La vida sense la Sara Amat,6.6,5,None


In [25]:
type(ratings_df['genres'][0])

str

In [28]:
ratings_df['split_genres'] = ratings_df['genres'].str.split(',')

In [31]:
ratings_df = ratings_df.dropna(subset = ['genres'])

In [34]:
ratings_df.explode('split_genres')

,primary_title,averagerating,numvotes,genres,split_genres
0,Sunghursh,7.0,77,"Action,Crime,Drama",Action
0,Sunghursh,7.0,77,"Action,Crime,Drama",Crime
0,Sunghursh,7.0,77,"Action,Crime,Drama",Drama
1,One Day Before the Rainy Season,7.2,43,"Biography,Drama",Biography
1,One Day Before the Rainy Season,7.2,43,"Biography,Drama",Drama
...,...,...,...,...,...
73851,Diabolik sono io,6.2,6,Documentary,Documentary
73852,Sokagin Çocuklari,8.7,136,"Drama,Family",Drama
73852,Sokagin Çocuklari,8.7,136,"Drama,Family",Family
73853,Albatross,8.5,8,Documentary,Documentary


In [46]:
x_df = ratings_df.groupby('genres').count().iloc[:,0]
x2_df = x_df[x_df >= 100].index

In [47]:
ratings_df[ratings_df.genres.isin(x2_df)]

,primary_title,averagerating,numvotes,genres,split_genres
0,Sunghursh,7.0,77,"Action,Crime,Drama","[Action, Crime, Drama]"
1,One Day Before the Rainy Season,7.2,43,"Biography,Drama","[Biography, Drama]"
2,The Other Side of the Wind,6.9,4517,Drama,[Drama]
3,Sabse Bada Sukh,6.1,13,"Comedy,Drama","[Comedy, Drama]"
4,The Wandering Soap Opera,6.5,119,"Comedy,Drama,Fantasy","[Comedy, Drama, Fantasy]"
...,...,...,...,...,...
73850,Swarm Season,6.2,5,Documentary,[Documentary]
73851,Diabolik sono io,6.2,6,Documentary,[Documentary]
73852,Sokagin Çocuklari,8.7,136,"Drama,Family","[Drama, Family]"
73853,Albatross,8.5,8,Documentary,[Documentary]


Creating data frame for Rotten Tomatoes data

In [9]:
cur.execute("""SELECT movie_title, genres, tomatometer_status, tomatometer_rating, tomatometer_count, audience_status, audience_rating, audience_count
               FROM rotten_tomatoes_movies;""")
rotten_tomatoes_df = pd.DataFrame(cur.fetchall())
rotten_tomatoes_df.columns = [x[0] for x in cur.description]
rotten_tomatoes_df.head(5)

,movie_title,genres,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count
0,Percy Jackson & the Olympians: The Lightning T...,"Action & Adventure, Comedy, Drama, Science Fic...",Rotten,49.0,149.0,Spilled,53.0,254421.0
1,Please Give,Comedy,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0
2,10,"Comedy, Romance",Fresh,67.0,24.0,Spilled,53.0,14684.0
3,12 Angry Men (Twelve Angry Men),"Classics, Drama",Certified-Fresh,100.0,54.0,Upright,97.0,105386.0
4,"20,000 Leagues Under The Sea","Action & Adventure, Drama, Kids & Family",Fresh,89.0,27.0,Upright,74.0,68918.0


In [10]:
type(rotten_tomatoes_df['genres'][0])

str